In [ ]:
#!pip install openai python-dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import openai
import re
import requests
import sys
import os
import pandas as pd
import numpy as np
#from dotenv import load_dotenv
#load_dotenv()

openai.api_type = "azure"
openai.api_version = "2022-12-01"

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT
#COMPLETIONS_MODEL = "davinci-search-query" # os.getenv('DEPLOYMENT_NAME')


### Use the utils to get a list of OpenAI Models deployed

In [ ]:
from utils import list_oai_models

sub_id = os.getenv("mysubscription")
rg = os.getenv("resource_group")
svc = os.getenv("azure_openai_instance")

df = list_oai_models(sub_id, rg, svc)

#### Display the Models

In [ ]:
df.head()

In [ ]:
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""
try:
    for n in df.index:
        response = openai.Completion.create(
            prompt=prompt,
            temperature=0,
            max_tokens=1000,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            engine=df.loc[n,"display_name"]
        )["choices"][0]["text"].strip(" \n")

        print(df.loc[n, "display_name"])
        print(response)
except Exception as e:
    print(e)

In [ ]:
prompt = """Decide whether the following customer feedback is positive or negative.

Q: I was disappointed with the quality of the product. It was very cheaply made and did not meet my expectations at all.
Q: I was happy with this product, it is well made and great quality for the price.
"""

try:
    for n in df.index:
        response = openai.Completion.create(
            prompt=prompt,
            temperature=0,
            max_tokens=300,
            engine=df.loc[n,"display_name"]
        )["choices"][0]["text"].strip(" \n")

        print(df.loc[n, "display_name"])
        print(response)
except Exception as e:
    print(e)

In [ ]:
prompt = """List all PII data from following statement:
John Doe is a 35-year old man and he lives at 21 Main Street, New York, NY. He is a software engineer and he works at Google. He has a wife named Jane Doe and they have two children
"""

try:
    for n in df.index:
        response = openai.Completion.create(
            prompt=prompt,
            temperature=0,
            max_tokens=300,
            engine=df.loc[n, "display_name"]
        )["choices"][0]["text"].strip(" \n")
        
        print(df.loc[n, "display_name"])
        print(f'\t{response}')
except Exception as e:
    print(e)